In [1]:
import pandas as pd
import json

In [2]:
part_pro_df = pd.read_csv("part_pro_df.csv")
metadata_copy = pd.read_csv("metadata.csv")
org_pro_df = pd.read_csv("org_pro_df.csv")
clubs_pro_df = pd.read_csv("clubs_pro_df.csv")

In [3]:
unique_fests = list(part_pro_df["Fest_Name"].unique())
unique_clubs = list(clubs_pro_df["Club_Name"].unique())

In [4]:
events_clubs = {}

for k in unique_clubs:
    events_clubs[k] = []

In [5]:
club_events_all = list(clubs_pro_df["Event"].unique())

for ce in club_events_all:
    for k in events_clubs:
        if ce.startswith(k):
            events_clubs[k].append(ce)

In [6]:
unique_fevents = list(part_pro_df["Event"].unique())

## Code for converting into a json Profile for each unique ID:

In [7]:
main_list = []

for i in range(len(metadata_copy)):
    
    curr_id = metadata_copy["ID"][i]
    curr_name = metadata_copy["Name"][i]

    dict1 = {}
        
    dict1["name"] = curr_name
    dict1["ID"] = curr_id
    # clubs -> dict
    club_dict = {}
    club_names = set(list(clubs_pro_df[clubs_pro_df["RollNumber"] == curr_id]["Club_Name"]))
    
    for cn in club_names:
        
        if cn not in club_dict:
            dict_club = {}   # For club i (dict inside clubi)
            club_dict[cn] = dict_club
            
            
            df1 = clubs_pro_df[clubs_pro_df["RollNumber"] == curr_id]
            df2 = df1[df1["Club_Name"] == cn]["Role"] != "Participant"
            df2 = df2[df2 == True]
            
            if len(df2) > 0:
                isOrg = True
                
            else:
                isOrg = False
 
             
        if isOrg == True:
            dict_club["isOrganiser"] = "Organiser"
            
        if isOrg == False:
            dict_club["isOrganiser"] = ""
            
            
        curr_club_events = list(set((clubs_pro_df[(clubs_pro_df["RollNumber"] == curr_id) & (clubs_pro_df["Club_Name"] == cn)]["Event"])))
        
        possible_club_events = []
        for e in club_events_all:
            if e.startswith(cn):
                possible_club_events.append(e)

        dict_club_events = {}
        for ce in possible_club_events:
            dict_club_events[ce] = {"participated":False}

        for pce in possible_club_events:
            if pce in curr_club_events:
                    
                df1 = clubs_pro_df[clubs_pro_df["RollNumber"] == curr_id]
                df2 = df1[df1["Club_Name"] == cn]
                df2 = df2[df2["Event"] == pce]
                
                if df2.iloc[0]["Role"] == "Participant":
                    dict_club_events[pce]["participated"] = True
                
                else:
                    dict_club_events[pce]["participated"] = False
                
#                 print(df2)
#                 print()
#                 df2 = df2[df2 == True]
                
#                 if len(df2) > 0:
#                     dict_club_events[pce]["participated"] = False

#                 else:
#                     dict_club_events[pce]["participated"] = True

        for k in dict_club_events:
            dict_club[k] = dict_club_events[k]

              
        club_dict[cn] = dict_club

    dict1["clubs"] = club_dict
    
    
    #############################################################
    
    
    fest_dict = {}
    fest_names = set(list(part_pro_df[part_pro_df["RollNumber"] == curr_id]["Fest_Name"]))
    
    for fn in fest_names:
        
        if fn not in fest_dict:
            dict_fest = {}   # For fest i (dict inside clubi)
            fest_dict[fn] = dict_fest
            
            
        if curr_id in org_pro_df["RollNumber"]:
            isOrg = True
                
        else:
            isOrg = False
            
            
        if isOrg == True:
            dict_fest["isOrganiser"] = "Organiser"
            
        if isOrg == False:
            dict_fest["isOrganiser"] = ""
            
            
        
        curr_fest_events = list(set((part_pro_df[(part_pro_df["RollNumber"] == curr_id) & (part_pro_df["Fest_Name"] == fn)]["Event"])))
        
        possible_fest_events = []
        for f in unique_fevents:
            if f.startswith(fn):
                possible_fest_events.append(f)
        
        dict_fest_events = {}
        for fe in possible_fest_events:
            dict_fest_events[fe] = {"participated":False}

        for fce in possible_fest_events:
            if fce in curr_fest_events:
                    
                df1 = part_pro_df[part_pro_df["RollNumber"] == curr_id]
                df2 = df1[df1["Fest_Name"] == fn]
                df2 = df2[df2["Event"] == fce]
                
                if len(df2) > 0:
                    dict_fest_events[fce]["participated"] = True
                    
                else:
                    dict_fest_events[fce]["participated"] = False
    
    
        for k in dict_fest_events:
            dict_fest[k] = dict_fest_events[k]

              
        fest_dict[fn] = dict_fest

    dict1["fests"] = fest_dict
    
    
    main_list.append(dict1)
    
    
            

In [8]:
main_list[2]

{'name': 'Jaquelin Duran',
 'ID': '18XJ1A0102',
 'clubs': {'club_1': {'isOrganiser': '',
   'club_1_event_2': {'participated': True},
   'club_1_event_1': {'participated': False},
   'club_1_event_3': {'participated': False}},
  'club_2': {'isOrganiser': '',
   'club_2_event_1': {'participated': True},
   'club_2_event_3': {'participated': False},
   'club_2_event_2': {'participated': False}},
  'club_3': {'isOrganiser': '',
   'club_3_event_2': {'participated': True},
   'club_3_event_1': {'participated': False},
   'club_3_event_3': {'participated': False}}},
 'fests': {'fest_1': {'isOrganiser': '',
   'fest_1_event_9': {'participated': False},
   'fest_1_event_10': {'participated': False},
   'fest_1_event_8': {'participated': False},
   'fest_1_event_4': {'participated': True},
   'fest_1_event_6': {'participated': False},
   'fest_1_event_11': {'participated': False},
   'fest_1_event_2': {'participated': False},
   'fest_1_event_12': {'participated': False},
   'fest_1_event_3': 

In [14]:
final = json.dumps(main_list, indent=2)

In [15]:
import json

with open("student_profiles.json", "w") as write_file:
    json.dump(final, write_file, indent=2)